# 11.4 Recap

In [2]:
import subprocess
from pydub import AudioSegment
import math
import glob
import openai

def extract_audio_from_video(video_path, audio_path):
    command = ["ffmpeg", "-i", video_path, "-vn", audio_path,]
    # 이렇게 배열로 형태로 전달하는 것은 보안때문이다. 또한 특수 문자가 포함되어 있을때도 더 편리하기 때문에
    # 배열방식이 권장된다.
    subprocess.run(command)

# extract_audio_from_video("../files/Joe_Dispenza_podcast.mp4", "../files/audio.mp3")

def cut_audio_in_chunks(audio_path, chunk_size, chunks_folder):
    track = AudioSegment.from_mp3(audio_path)
    chunk_len = chunk_size * 60 * 1000 # 5분을 밀리초 단위로 변환
    chunks = math.ceil(len(track) / chunk_len) # ceil은 올림 할때 쓰는 함수

    for i in range(chunks):
        start_time = i * chunk_len
        end_time = (i+1) * chunk_len

        if i < chunks-1:
            splited_track = track[start_time:end_time]
            splited_track.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")
        else:
            splited_track = track[start_time:]
            splited_track.export(f"{chunks_folder}/chunk_{i}.mp3", format="mp3")

# cut_audio_in_chunks("../files/audio.mp3", 10, "../files/chunks")


일단 먼저 이전에 만들었던 함수를 조금 수정해보자.  
아래와 같이 수정해 보았다. 좀 더 간결하다.

In [ ]:
client = openai.OpenAI()

def transcribe_chunks(chunk_folder, destination):
    files = glob.glob(f"{chunk_folder}/*.mp3")
    for file in files:
        with open(file, "rb") as audio_file, open(destination, "a") as text_file:
            transcript = client.audio.transcriptions.create(
                model="whisper-1", 
                file=audio_file,
            )
            text_file.write(transcript.text)

# transcribe_chunks("../files/chunks", "../files/transcript.txt")